In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize, ToTensor, Compose, Resize
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

/Users/tedsoderberg/anaconda3/envs/image-discriminator/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/tedsoderberg/anaconda3/envs/image-discriminator/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <E03EDA44-89AE-3115-9796-62BA9E0E2EDE> /Users/tedsoderberg/anaconda3/envs/image-discriminator/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F75BB06B-2723-344D-99CE-9CB8BB94077A> /Users/tedsoderberg/anaconda3/envs/image-discriminator/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
#Hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [14]:
data_transforms = {
    'train': Compose([
        Resize((28, 28)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': Compose([
        Resize((28, 28)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dir = 'archive/train'
test_dir = 'archive/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train']) #100 000 pictures
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)



In [5]:
def imgshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg), (1,2,0))
    plt.show()

In [8]:

class ConvNet(nn.Module):
    #(W - K + 2P)/S + 1
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #(input channel, output channel, kernal size) #dim = 24
        self.pool = nn.MaxPool2d(2,2) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [9]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)


In [11]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad() #reset all weights
        outputs = model(images) 
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
        if (i+1) % 10000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}],Step [{i+1}/{n_total_steps}] Loss: {loss.item():.4f}')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("Finished Training!")



Epoch [1/4],Step [10000/25000] Loss: 2.1561
Epoch [1/4],Step [20000/25000] Loss: 1.7013
Epoch [1/4], Loss: 0.3075
Epoch [2/4],Step [10000/25000] Loss: 0.1814
Epoch [2/4],Step [20000/25000] Loss: 2.5437
Epoch [2/4], Loss: 2.2227
Epoch [3/4],Step [10000/25000] Loss: 2.2882
Epoch [3/4],Step [20000/25000] Loss: -0.0000
Epoch [3/4], Loss: 2.2201
Epoch [4/4],Step [10000/25000] Loss: 1.1997
Epoch [4/4],Step [20000/25000] Loss: 3.9053
Epoch [4/4], Loss: 1.5439
Finished Training!


In [15]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.round(torch.sigmoid(outputs.squeeze()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Accuracy of the model on the test images: 83.75 %
